# Clustering Toronto Neighbourhoods
## _The battle of the Neighbourhoods_

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values (OpenStreetMaps)
import requests
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [2]:
df = pd.read_csv('toronto_geocoded_postcodes.csv', index_col=0)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440


In [3]:
print('Toronto summary : \n There are {} unique Postcodes (Aggregated Neighbourhoods) \
and \n {} Boroughs in the Toronto DataFrame'.format(df.Postcode.unique().size,df.Borough.unique().size))

Toronto summary : 
 There are 103 unique Postcodes (Aggregated Neighbourhoods) and 
 11 Boroughs in the Toronto DataFrame


In [4]:
df.shape

(103, 5)

In [5]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [6]:
neighborhoods = df
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.4)

# add markers to map
for lat, lng, borough, neighbourhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [7]:
#FourSquare loading
CLIENT_ID,CLIENT_SECRET,VERSION = pd.read_csv('foursquare_api_keys.csv')
print(neighborhoods.head())
print(neighborhoods.shape)

  Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.811650   
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.785605   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.765690   
3      M1G  Scarborough                                  Woburn  43.768216   
4      M1H  Scarborough                               Cedarbrae  43.769608   

   Longitude  
0 -79.195561  
1 -79.158701  
2 -79.175299  
3 -79.217610  
4 -79.239440  
(103, 5)


In [8]:
def getBuiltUrl(neigh_lat,neigh_long,radius=1400):
    # type your answer here
    LIMIT=100
    #radius=1000
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_long, 
    radius, 
    LIMIT)
    return url
    #url

In [9]:
# get one test Neighbourhood
neigh_post, neigh_borough, neigh_name, neigh_lat, neigh_long = df.iloc[0]
print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_long))
results = requests.get(getBuiltUrl(neigh_lat,neigh_long)).json()
results

Latitude and longitude values of Rouge, Malvern are 43.811650000000036, -79.19556138899999.


{'meta': {'code': 200, 'requestId': '5c265347351e3d5f4f1b7e8f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 26,
  'suggestedBounds': {'ne': {'lat': 43.82425001260005,
    'lng': -79.17813324674674},
   'sw': {'lat': 43.799049987400025, 'lng': -79.21298953125324}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c97975582b56dcb8320ebaa',
       'name': 'Canadiana exhibit',
       'location': {'lat': 43.81796218928876,
        'lng': -79.19337359666939,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.81796218928876,
          'lng': -79.1933

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# clean the json

In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Canadiana exhibit,Zoo Exhibit,43.817962,-79.193374
1,African Rainforest Pavilion,Zoo Exhibit,43.817725,-79.183433
2,Lion Exhibit,Zoo Exhibit,43.819228,-79.186977
3,penguin exhibit,Zoo Exhibit,43.819435,-79.185959
4,Gorilla Exhibit,Zoo Exhibit,43.819080,-79.184235


In [12]:
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4c97975582b56dcb8320ebaa',
   'name': 'Canadiana exhibit',
   'location': {'lat': 43.81796218928876,
    'lng': -79.19337359666939,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.81796218928876,
      'lng': -79.19337359666939}],
    'distance': 724,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Toronto ON', 'Canada']},
   'categories': [{'id': '58daa1558bbb0b01f18ec1fd',
     'name': 'Zoo Exhibit',
     'pluralName': 'Zoo Exhibits',
     'shortName': 'Zoo Exhibit',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/zoo_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4c97975582b56dcb8320ebaa-0'},
 {'reasons': {'count': 0,
   'items': [{'summary': 'Th

In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

26 venues were returned by Foursquare.


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = getBuiltUrl(lat,lng,radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
#neighborhoods = neighborhoods.drop(neighborhoods.columns[0], axis=1)
neighborhoods['Neighbourhood'] = neighborhoods['Neighbourhood']+' '+neighborhoods['Postcode']


neighborhoods.head()
neighborhoods['Neighbourhood'].size

103

In [16]:

toronto_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'], 
                                   latitudes=neighborhoods['Latitude'], 
                                   longitudes=neighborhoods['Longitude'], 
                                   radius=4000)

Rouge, Malvern M1B
Highland Creek, Rouge Hill, Port Union M1C
Guildwood, Morningside, West Hill M1E
Woburn M1G
Cedarbrae M1H
Scarborough Village M1J
East Birchmount Park, Ionview, Kennedy Park M1K
Clairlea, Golden Mile, Oakridge M1L
Cliffcrest, Cliffside, Scarborough Village West M1M
Birch Cliff, Cliffside West M1N
Dorset Park, Scarborough Town Centre, Wexford Heights M1P
Maryvale, Wexford M1R
Agincourt M1S
Clarks Corners, Sullivan, Tam O'Shanter M1T
Agincourt North, L'Amoreaux East, Milliken, Steeles East M1V
L'Amoreaux West, Steeles West M1W
Upper Rouge M1X
Hillcrest Village M2H
Fairview, Henry Farm, Oriole M2J
Bayview Village M2K
Silver Hills, York Mills M2L
Newtonbrook, Willowdale M2M
Willowdale South M2N
York Mills West M2P
Willowdale West M2R
Parkwoods M3A
Don Mills North M3B
Flemingdon Park, Don Mills South M3C
Bathurst Manor, Downsview North, Wilson Heights M3H
Northwood Park, York University M3J
CFB Toronto, Downsview East M3K
Downsview West M3L
Downsview Central M3M
Downsview

In [17]:
print(toronto_venues.shape)
toronto_venues.head()


(10219, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern M1B",43.81165,-79.195561,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
1,"Rouge, Malvern M1B",43.81165,-79.195561,Toronto Zoo,43.820582,-79.181551,Zoo
2,"Rouge, Malvern M1B",43.81165,-79.195561,Polar Bear Exhibit,43.823372,-79.185145,Zoo
3,"Rouge, Malvern M1B",43.81165,-79.195561,Canadiana exhibit,43.817962,-79.193374,Zoo Exhibit
4,"Rouge, Malvern M1B",43.81165,-79.195561,Australasia Pavillion,43.822563,-79.183286,Zoo Exhibit


In [18]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond M5H",100,100,100,100,100,100
Agincourt M1S,100,100,100,100,100,100
"Agincourt North, L'Amoreaux East, Milliken, Steeles East M1V",100,100,100,100,100,100
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown M9V",100,100,100,100,100,100
"Alderwood, Long Branch M8W",100,100,100,100,100,100
"Bathurst Manor, Downsview North, Wilson Heights M3H",100,100,100,100,100,100
Bayview Village M2K,100,100,100,100,100,100
"Bedford Park, Lawrence Manor East M5M",100,100,100,100,100,100
Berczy Park M5E,100,100,100,100,100,100


In [19]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print('There are {} uniques Neighborhoods.'.format(len(toronto_venues['Neighborhood'].unique())))

There are 257 uniques categories.
There are 103 uniques Neighborhoods.


In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
#toronto_venues['Neighborhood'].size

,Zoo Exhibit,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge, Malvern M1B",

In [21]:
toronto_onehot.shape

(10219, 257)

In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Adelaide, King, Richmond M5H",0.000000,0.01,0.000000,0.00,0.010000,0.01,0.00,0.02,0.00,0.000000,0.000000,0.00,0.01,0.00,0.000000,0.000000,0.00,0.000000,0.02,0.00,0.01,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.010000,0.00,0.00,0.00,0.010000,0.01,0.00,0.00,0.00,0.000000,0.00,0.000000,0.01,0.00,0.000000,0.04,0.000000,0.00,0.000000,0.00,0.00,0.00000

In [23]:
toronto_grouped.shape

(103, 257)

In [24]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond M5H----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.05
2               Hotel  0.04
3                Café  0.04
4      Sandwich Place  0.03


----Agincourt M1S----
                  venue  freq
0    Chinese Restaurant  0.10
1           Coffee Shop  0.07
2          Noodle House  0.05
3  Caribbean Restaurant  0.05
4     Indian Restaurant  0.04


----Agincourt North, L'Amoreaux East, Milliken, Steeles East M1V----
                  venue  freq
0    Chinese Restaurant  0.18
1           Coffee Shop  0.04
2       Bubble Tea Shop  0.04
3  Caribbean Restaurant  0.04
4          Dessert Shop  0.03


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown M9V----
                  venue  freq
0           Coffee Shop  0.17
1  Fast Food Restaurant  0.10
2        Sandwich Place  0.06
3           Pizza Place  0.06
4                  Bank  0.05


----Alderwood, Long Branch M8W----

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond M5H",Coffee Shop,Italian Restaurant,Café,Hotel,Japanese Restaurant,Park,Theater,Sandwich Place,Baseball Stadium,Pizza Place
1,Agincourt M1S,Chinese Restaurant,Coffee Shop,Noodle House,Caribbean Restaurant,Indian Restaurant,Supermarket,Clothing Store,Sushi Restaurant,Pharmacy,Steakhouse
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Caribbean Restaurant,Bubble Tea Shop,Coffee Shop,Vietnamese Restaurant,Indian Restaurant,Bakery,Noodle House,Dessert Shop,Sushi Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Fast Food Restaurant,Sandwich Place,Pizza Place,Indian Restaurant,Bank,Grocery Store,Caribbean Restaurant,Pharmacy,Asian Restaurant
4,"Alderwood, Long Branch M8W",Coffee Shop,Burger Joint,Bakery,Café,Seafood Restaurant,Burrito Place,Park,Middle Eastern Restaurant,Pizza Place,Restaurant
5,"Bathurst Manor, Downsview North, Wilson Height...",Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Bagel Shop,Movie Theater,Korean Restaurant,Sushi Restaurant,Bookstore,Gym
6,Bayview Village M2K,Korean Restaurant,Supermarket,Coffee Shop,Bakery,Sushi Restaurant,Chinese Restaurant,Grocery Store,Café,Burger Joint,Middle Eastern Restaurant
7,"Bedford Park, Lawrence Manor East M5M",Coffee Shop,Café,Sushi Restaurant,Bakery,Clothing Store,Italian Restaurant,Cosmetics Shop,Bagel Shop,Park,Jewelry Store
8,Berczy Park M5E,Coffee Shop,Hotel,Café,Park,Farmers Market,Italian Restaurant,Japanese Restaurant,Thai Restaurant,Theater,Restaurant
9,"Birch Cliff, Cliffside West M1N",Sandwich Place,Coffee Shop,Park,Pub,Beach,Breakfast Spot,Burger Joint,Bank,Fast Food Restaurant,Japanese Restaurant


# Clustering

In [27]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#toronto_grouped_clustering.head()

array([3, 5, 5, 2, 6, 7, 4, 9, 3, 6], dtype=int32)

In [28]:
toronto_merged = neighborhoods#.merge(toronto_grouped['Neighborhood'], how='right', on='Neighborhood')
toronto_merged = toronto_merged.rename(columns={'Neighbourhood':'Neighborhood'})

In [29]:

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern M1B",43.811650,-79.195561,3,Zoo Exhibit,Coffee Shop,Fast Food Restaurant,Pharmacy,Pizza Place,Sandwich Place,Park,Burger Joint,Discount Store,Zoo
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union M1C",43.785605,-79.158701,5,Coffee Shop,Fast Food Restaurant,Park,Pizza Place,Sandwich Place,Breakfast Spot,Burger Joint,Pub,Beer Store,Fried Chicken Joint
2,M1E,Scarborough,"Guildwood, Morningside, West Hill M1E",43.765690,-79.175299,5,Coffee Shop,Park,Sandwich Place,Fast Food Restaurant,Pizza Place,Burger Joint,Supermarket,Beer Store,Pharmacy,Grocery Store
3,M1G,Scarborough,Woburn M1G,43.768216,-79.217610,2,Coffee Shop,Caribbean Restaurant,Indian Restaurant,Gym,Clothing Store,Bank,Pharmacy,Pizza Place,Park,Sandwich Place
4,M1H,Scarborough,Cedarbrae M1H,43.769608,-79.239440,6,Coffee Shop,Chinese Restaurant,Indian Restaurant,Gym,Clothing Store,Caribbean Restaurant,Bookstore,Supermarket,Bakery,Pharmacy


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine clusters

## cluster 0

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,0,Middle Eastern Restaurant,Coffee Shop,Burger Joint,Supermarket,Indian Restaurant,Chinese Restaurant,Seafood Restaurant,Sandwich Place,Caribbean Restaurant,Pet Store
16,Scarborough,0,Zoo Exhibit,Fast Food Restaurant,Coffee Shop,Sandwich Place,Grocery Store,Pharmacy,Pizza Place,Hakka Restaurant,Sports Bar,Burger Joint
21,North York,0,Korean Restaurant,Coffee Shop,Middle Eastern Restaurant,Chinese Restaurant,Sushi Restaurant,Grocery Store,Café,Supermarket,Burrito Place,Burger Joint
35,East York,0,Bakery,Gastropub,Middle Eastern Restaurant,Café,Coffee Shop,Indian Restaurant,Turkish Restaurant,Thai Restaurant,Gym / Fitness Center,Brewery
51,Downtown Toronto,0,Coffee Shop,Café,Park,Farmers Market,Japanese Restaurant,Italian Restaurant,Gastropub,Breakfast Spot,Diner,Thai Restaurant
65,Central Toronto,0,Café,Bar,Cocktail Bar,Coffee Shop,Sandwich Place,Vegetarian / Vegan Restaurant,Indian Restaurant,Grocery Store,Italian Restaurant,Park
73,York,0,Café,Coffee Shop,Italian Restaurant,Bar,Spa,Park,Bakery,Ice Cream Shop,Indian Restaurant,Mexican Restaurant
79,North York,0,Clothing Store,Coffee Shop,Liquor Store,Furniture / Home Store,Bakery,Vietnamese Restaurant,Falafel Restaurant,Jewelry Store,Men's Store,Italian Restaurant
86,Mississauga,0,Coffee Shop,Hotel,Café,Italian Restaurant,Park,Restaurant,Theater,Sandwich Place,Gastropub,Concert Hall
90,Etobicoke,0,Café,Coffee Shop,Bakery,Italian Restaurant,Bar,Burger Joint,Ice Cream Shop,Grocery Store,Pizza Place,Liquor Store


## cluster 1

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,East York,1,Café,Greek Restaurant,Park,Brewery,Coffee Shop,Pizza Place,Bakery,Gastropub,Indian Restaurant,Farmers Market
85,Queen's Park,1,Café,Coffee Shop,Sandwich Place,Restaurant,Japanese Restaurant,Hotel,Vegetarian / Vegan Restaurant,Italian Restaurant,Theater,Pizza Place
87,East Toronto,1,Coffee Shop,Hotel,Café,Italian Restaurant,Park,Restaurant,Theater,Sandwich Place,Gastropub,Concert Hall
88,Etobicoke,1,Park,Coffee Shop,Italian Restaurant,Sushi Restaurant,Bakery,Café,Thai Restaurant,Liquor Store,Middle Eastern Restaurant,Pizza Place
89,Etobicoke,1,Coffee Shop,Burger Joint,Bakery,Café,Seafood Restaurant,Burrito Place,Park,Middle Eastern Restaurant,Pizza Place,Restaurant
92,Etobicoke,1,Coffee Shop,Burger Joint,Sushi Restaurant,Bakery,Grocery Store,Seafood Restaurant,Furniture / Home Store,Pizza Place,Dessert Shop,Café
101,Etobicoke,1,Coffee Shop,Fast Food Restaurant,Sandwich Place,Pizza Place,Indian Restaurant,Bank,Grocery Store,Caribbean Restaurant,Pharmacy,Asian Restaurant


# Cluster 1

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,East York,1,Café,Greek Restaurant,Park,Brewery,Coffee Shop,Pizza Place,Bakery,Gastropub,Indian Restaurant,Farmers Market
85,Queen's Park,1,Café,Coffee Shop,Sandwich Place,Restaurant,Japanese Restaurant,Hotel,Vegetarian / Vegan Restaurant,Italian Restaurant,Theater,Pizza Place
87,East Toronto,1,Coffee Shop,Hotel,Café,Italian Restaurant,Park,Restaurant,Theater,Sandwich Place,Gastropub,Concert Hall
88,Etobicoke,1,Park,Coffee Shop,Italian Restaurant,Sushi Restaurant,Bakery,Café,Thai Restaurant,Liquor Store,Middle Eastern Restaurant,Pizza Place
89,Etobicoke,1,Coffee Shop,Burger Joint,Bakery,Café,Seafood Restaurant,Burrito Place,Park,Middle Eastern Restaurant,Pizza Place,Restaurant
92,Etobicoke,1,Coffee Shop,Burger Joint,Sushi Restaurant,Bakery,Grocery Store,Seafood Restaurant,Furniture / Home Store,Pizza Place,Dessert Shop,Café
101,Etobicoke,1,Coffee Shop,Fast Food Restaurant,Sandwich Place,Pizza Place,Indian Restaurant,Bank,Grocery Store,Caribbean Restaurant,Pharmacy,Asian Restaurant


## cluster 2

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,2,Coffee Shop,Caribbean Restaurant,Indian Restaurant,Gym,Clothing Store,Bank,Pharmacy,Pizza Place,Park,Sandwich Place
25,North York,2,Middle Eastern Restaurant,Coffee Shop,Japanese Restaurant,Mediterranean Restaurant,Supermarket,Burger Joint,Italian Restaurant,Café,Park,Shopping Mall
37,East Toronto,2,Park,Coffee Shop,Café,Beach,Breakfast Spot,Bakery,Gastropub,Brewery,Pub,BBQ Joint
38,East York,2,Park,Café,Coffee Shop,Italian Restaurant,Bakery,Grocery Store,Gym,Supermarket,Restaurant,Indian Restaurant
41,East Toronto,2,Park,Coffee Shop,Italian Restaurant,Greek Restaurant,Café,Diner,Japanese Restaurant,Pizza Place,Restaurant,Vietnamese Restaurant
47,Central Toronto,2,Park,Italian Restaurant,Coffee Shop,Café,Bakery,Gastropub,Indian Restaurant,Bookstore,Spa,Sushi Restaurant
52,Downtown Toronto,2,Coffee Shop,Café,Japanese Restaurant,Park,Italian Restaurant,Gastropub,Theater,Restaurant,Vegetarian / Vegan Restaurant,Concert Hall
56,Downtown Toronto,2,Coffee Shop,Hotel,Café,Park,Farmers Market,Italian Restaurant,Japanese Restaurant,Thai Restaurant,Theater,Restaurant
59,Downtown Toronto,2,Park,Italian Restaurant,Coffee Shop,Bakery,Hotel,Gym,Pizza Place,Theater,Bookstore,Spa
71,North York,2,Clothing Store,Bakery,Coffee Shop,Furniture / Home Store,Café,Bagel Shop,Falafel Restaurant,Italian Restaurant,Athletics & Sports,Deli / Bodega


## cluster 3

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3,Zoo Exhibit,Coffee Shop,Fast Food Restaurant,Pharmacy,Pizza Place,Sandwich Place,Park,Burger Joint,Discount Store,Zoo
8,Scarborough,3,Coffee Shop,Sandwich Place,Park,Fast Food Restaurant,Bank,Pharmacy,Pizza Place,Chinese Restaurant,Fried Chicken Joint,Gym
12,Scarborough,3,Chinese Restaurant,Coffee Shop,Noodle House,Caribbean Restaurant,Indian Restaurant,Supermarket,Clothing Store,Sushi Restaurant,Pharmacy,Steakhouse
14,Scarborough,3,Chinese Restaurant,Caribbean Restaurant,Bubble Tea Shop,Coffee Shop,Vietnamese Restaurant,Indian Restaurant,Bakery,Noodle House,Dessert Shop,Sushi Restaurant
15,Scarborough,3,Chinese Restaurant,Bakery,Caribbean Restaurant,Japanese Restaurant,Bubble Tea Shop,Asian Restaurant,Noodle House,Tea Room,Korean Restaurant,Italian Restaurant
17,North York,3,Chinese Restaurant,Coffee Shop,Japanese Restaurant,Bakery,Caribbean Restaurant,Asian Restaurant,Sushi Restaurant,Sporting Goods Shop,Supermarket,Electronics Store
19,North York,3,Korean Restaurant,Supermarket,Coffee Shop,Bakery,Sushi Restaurant,Chinese Restaurant,Grocery Store,Café,Burger Joint,Middle Eastern Restaurant
20,North York,3,Burger Joint,Bakery,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Bubble Tea Shop,Shopping Mall,Seafood Restaurant,Middle Eastern Restaurant,Café
22,North York,3,Korean Restaurant,Coffee Shop,Café,Burger Joint,Middle Eastern Restaurant,Sushi Restaurant,Bakery,Grocery Store,Seafood Restaurant,Burrito Place
27,North York,3,Bakery,Middle Eastern Restaurant,Park,Supermarket,Grocery Store,Gym / Fitness Center,Indian Restaurant,Japanese Restaurant,Coffee Shop,Ice Cream Shop


## cluster 4

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,4,Coffee Shop,Burger Joint,Middle Eastern Restaurant,Chinese Restaurant,Park,Indian Restaurant,Pharmacy,Supermarket,Fast Food Restaurant,Beach
69,Downtown Toronto,4,Coffee Shop,Hotel,Café,Italian Restaurant,Park,Restaurant,Theater,Sandwich Place,Gastropub,Concert Hall
82,West Toronto,4,Café,Bar,Coffee Shop,Italian Restaurant,Gastropub,Park,Brewery,Bakery,Eastern European Restaurant,Restaurant
97,North York,4,Fast Food Restaurant,Coffee Shop,Pizza Place,Bank,Sandwich Place,Pharmacy,Fried Chicken Joint,Beer Store,Vietnamese Restaurant,Asian Restaurant
98,York,4,Coffee Shop,Sandwich Place,Fast Food Restaurant,Bank,Pizza Place,Pharmacy,Beer Store,Grocery Store,Breakfast Spot,Golf Course


## cluster 5


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,5,Coffee Shop,Fast Food Restaurant,Park,Pizza Place,Sandwich Place,Breakfast Spot,Burger Joint,Pub,Beer Store,Fried Chicken Joint
2,Scarborough,5,Coffee Shop,Park,Sandwich Place,Fast Food Restaurant,Pizza Place,Burger Joint,Supermarket,Beer Store,Pharmacy,Grocery Store
18,North York,5,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Bakery,Japanese Restaurant,Middle Eastern Restaurant,Seafood Restaurant,Supermarket,Burger Joint,Sushi Restaurant
24,North York,5,Korean Restaurant,Coffee Shop,Middle Eastern Restaurant,Sushi Restaurant,Grocery Store,Café,Restaurant,Ramen Restaurant,Japanese Restaurant,Supermarket
34,North York,5,Middle Eastern Restaurant,Burger Joint,Coffee Shop,Gym / Fitness Center,Supermarket,Japanese Restaurant,Bakery,Park,Dessert Shop,Café
42,East Toronto,5,Café,Coffee Shop,Park,Beach,Bakery,Brewery,Pizza Place,Gastropub,Ice Cream Shop,BBQ Joint
53,Downtown Toronto,5,Coffee Shop,Café,Hotel,Japanese Restaurant,Park,Gastropub,Italian Restaurant,Restaurant,Breakfast Spot,Thai Restaurant
61,Downtown Toronto,5,Coffee Shop,Café,Hotel,Park,Italian Restaurant,Japanese Restaurant,Gym,Farmers Market,Sandwich Place,Theater


## cluster 6

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,6,Coffee Shop,Chinese Restaurant,Indian Restaurant,Gym,Clothing Store,Caribbean Restaurant,Bookstore,Supermarket,Bakery,Pharmacy
9,Scarborough,6,Sandwich Place,Coffee Shop,Park,Pub,Beach,Breakfast Spot,Burger Joint,Bank,Fast Food Restaurant,Japanese Restaurant
10,Scarborough,6,Coffee Shop,Chinese Restaurant,Middle Eastern Restaurant,Supermarket,Indian Restaurant,Clothing Store,Grocery Store,Gym,Bookstore,Caribbean Restaurant
23,North York,6,Coffee Shop,Sushi Restaurant,Café,Park,Bagel Shop,Bakery,Burger Joint,Tea Room,Pub,Japanese Restaurant
26,North York,6,Bakery,Middle Eastern Restaurant,Japanese Restaurant,Burger Joint,Coffee Shop,Supermarket,Park,Shopping Mall,Furniture / Home Store,Ice Cream Shop
33,North York,6,Coffee Shop,Sandwich Place,Bank,Pizza Place,Fast Food Restaurant,Hotel,Pharmacy,Restaurant,Beer Store,Convenience Store
39,East York,6,Park,Café,Greek Restaurant,Grocery Store,Bakery,Gastropub,Brewery,Pizza Place,Ice Cream Shop,Coffee Shop
44,Central Toronto,6,Café,Bakery,Coffee Shop,Italian Restaurant,Park,Sandwich Place,Grocery Store,Tea Room,BBQ Joint,Asian Restaurant
54,Downtown Toronto,6,Coffee Shop,Café,Hotel,Japanese Restaurant,Pizza Place,Theater,Restaurant,Farmers Market,Gastropub,Park
58,Downtown Toronto,6,Coffee Shop,Italian Restaurant,Café,Hotel,Japanese Restaurant,Park,Theater,Sandwich Place,Baseball Stadium,Pizza Place


## cluster 7

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,7,Coffee Shop,Pharmacy,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Indian Restaurant,Bank,Beer Store,Park
13,Scarborough,7,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Breakfast Spot,Noodle House,Fast Food Restaurant,Vietnamese Restaurant,Indian Restaurant,Seafood Restaurant,Gym / Fitness Center
36,East York,7,Café,Park,Coffee Shop,Gastropub,Bakery,Beach,Pizza Place,Brewery,Greek Restaurant,Breakfast Spot
46,Central Toronto,7,Coffee Shop,Café,Bakery,Italian Restaurant,Park,Sushi Restaurant,Japanese Restaurant,Burger Joint,Gastropub,Breakfast Spot
62,North York,7,Coffee Shop,Café,Sushi Restaurant,Bakery,Clothing Store,Italian Restaurant,Cosmetics Shop,Bagel Shop,Park,Jewelry Store
66,Downtown Toronto,7,Café,Pizza Place,Sandwich Place,Vegetarian / Vegan Restaurant,Cocktail Bar,Coffee Shop,Park,Italian Restaurant,Bar,Grocery Store


## cluster 8

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
78,West Toronto,8,Bar,Café,Bakery,Italian Restaurant,Asian Restaurant,Sandwich Place,Pizza Place,Park,French Restaurant,Cocktail Bar
93,Etobicoke,8,Coffee Shop,Bakery,Pharmacy,Burger Joint,Pizza Place,Liquor Store,Italian Restaurant,Café,Grocery Store,Bank


## cluster 9

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,9,Middle Eastern Restaurant,Burger Joint,Coffee Shop,Breakfast Spot,Supermarket,Bank,Park,Gym / Fitness Center,Grocery Store,Indian Restaurant
28,North York,9,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Bagel Shop,Movie Theater,Korean Restaurant,Sushi Restaurant,Bookstore,Gym
29,North York,9,Coffee Shop,Pharmacy,Sandwich Place,Grocery Store,Restaurant,Pizza Place,Middle Eastern Restaurant,Breakfast Spot,Fast Food Restaurant,Hotel
30,North York,9,Clothing Store,Coffee Shop,Bakery,Deli / Bodega,Cosmetics Shop,Liquor Store,Boutique,Sporting Goods Shop,Café,Jewelry Store
31,North York,9,Coffee Shop,Pizza Place,Vietnamese Restaurant,Sandwich Place,Fast Food Restaurant,Pharmacy,Bank,Grocery Store,Middle Eastern Restaurant,Restaurant
45,Central Toronto,9,Coffee Shop,Café,Bakery,Park,Sushi Restaurant,Italian Restaurant,Gastropub,Burger Joint,Gym,Bookstore
55,Downtown Toronto,9,Coffee Shop,Japanese Restaurant,Hotel,Café,Gastropub,Theater,Restaurant,Farmers Market,Italian Restaurant,Park
57,Downtown Toronto,9,Coffee Shop,Café,Hotel,Sandwich Place,Concert Hall,Gastropub,Japanese Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Theater
63,Central Toronto,9,Coffee Shop,Bakery,Café,Italian Restaurant,Park,Burger Joint,Clothing Store,Japanese Restaurant,Toy / Game Store,Sushi Restaurant
64,Central Toronto,9,Italian Restaurant,Café,Coffee Shop,Park,Middle Eastern Restaurant,Bakery,Dessert Shop,Trail,Spa,BBQ Joint


# Thank you very much!